In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import zlib
import copy
import seaborn as sns
from scipy import signal, ndimage
from scipy.signal import spectrogram, butter, sosfilt

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from matplotlib import pyplot
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
import keras
import keras.utils
from keras import utils as np_utils

In [ ]:
from google.colab import drive
!unzip /content/human-activity-recognition-with-rnn.zip
train = pd.read_csv(r'/content/train.csv')
test = pd.read_csv(r'/content/test.csv')

Archive:  /content/human-activity-recognition-with-rnn.zip
replace activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
data = pd.read_csv(r'/content/test.csv')
train.head()

,id,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z",...,fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-std(),fBodyBodyGyroMag-mad(),fBodyBodyGyroMag-max(),fBodyBodyGyroMag-min(),fBodyBodyGyroMag-sma(),fBodyBodyGyroMag-energy(),fBodyBodyGyroMag-iqr(),fBodyBodyGyroMag-entropy(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroJerkMag-mad(),fBodyBodyGyroJerkMag-max(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-sma(),fBodyBodyGyroJerkMag-energy(),fBodyBodyGyroJerkMag-iqr(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,activity
0,0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.943220,-0.407747,-0.679338,-0.602122,0.929294,-0.853011,0.359910,-0.058526,0.256892,-0.224848,0.264106,-0.095246,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,...,-1.0,-0.936508,0.346989,-0.516080,-0.802760,-0.980135,-0.961309,-0.973653,-0.952264,-0.989498,-0.980135,-0.999240,-0.992656,-0.701291,-1.000000,-0.128989,0.586156,0.374605,-0.991990,-0.990697,-0.989941,-0.992448,-0.991048,-0.991990,-0.999937,-0.990458,-0.871306,-1.000000,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,5
1,1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,-0.557851,-0.818409,0.849308,0.685845,0.822637,-0.981930,-0.999991,-0.999788,-0.998405,-0.999150,-0.977866,-0.948225,-0.714892,-0.500930,-0.570979,0.611627,-0.329549,0.284213,0.284595,0.115705,-0.090963,0.294310,-0.281211,0.085988,-0.022153,-0.016657,-0.220643,-0.013429,-0.072692,...,-1.0,-0.841270,0.532061,-0.624871,-0.900160,-0.988296,-0.983322,-0.982659,-0.986321,-0.991829,-0.988296,-0.999811,-0.993979,-0.720683,-0.948718,-0.271958,-0.336310,-0.720015,-0.995854,-0.996399,-0.995442,-0.996866,-0.994440,-0.995854,-0.999981,-0.994544,-1.000000,-1.000000,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,5
2,2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,-0.557851,-0.818409,0.843609,0.682401,0.839344,-0.983478,-0.999969,-0.999660,-0.999470,-0.997130,-0.964810,-0.974675,-0.592235,-0.485821,-0.570979,0.273025,-0.086309,0.337202,-0.164739,0.017150,-0.074507,0.342256,-0.332564,0.239281,-0.136204,0.173863,-0.299493,-0.124698,-0.181105,...,-1.0,-0.904762,0.660795,-0.724697,-0.928539,-0.989255,-0.986028,-0.984274,-0.990979,-0.995703,-0.989255,-0.999854,-0.993238,-0.736521,-0.794872,-0.212728,-0.535352,-0.871914,-0.995031,-0.995127,-0.

In [ ]:
train=train.drop('id', axis=1)
print(train.shape)

(7352, 563)


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, tBodyAcc-mean()-X to activity
dtypes: float64(561), int64(2)
memory usage: 31.6 MB


In [ ]:
print(train["activity"].nunique())

6


In [ ]:
labels=['WALKING', 'WALKING_UPSTAIRS','WALKING_DOWNSTAIRS','SITTING','STANDING','LAYING']

In [ ]:
from keras.utils.np_utils import to_categorical
y=train['activity']
x=train.drop('activity',axis=1)
y=y-1
y = to_categorical(y, num_classes=6)
print(y.shape)
print(y[0])

(7352, 6)
[0. 0. 0. 0. 1. 0.]


In [ ]:
# from scipy import stats
# def create_dataset(X, y):
#     Xs, ys = [], []
#     start = 0 
#     max=0
#     for ind in range(1,len(X)):
#         if y[ind] != y[start]:
#             v = X[start:(ind-1)].values
#             labels = y[start: (ind-1)]
#             Xs.append(v)
#             ys.append(y[start])
#             if ind-start > max:
#               max = ind-start
#             start=ind
#     # for i in range(0, len(X) - time_steps, step):
#     #     v = X.iloc[i:(i + time_steps)].values
#     #     labels = y.iloc[i: i + time_steps]
#     #     Xs.append(v)
#     #     ys.append(stats.mode(labels)[0][0])
#     # print(max)

#     for i,row in enumerate(Xs):
#      result = np.zeros((max,562))
#      result[:Xs[i].shape[0],:Xs[i].shape[1]] = Xs[i]
#      Xs[i]=result
#     #   while len(Xs[i])<max:
#     #     print(Xs[i].shape)
#     #     # np.array(*np.array(Xs[i]),*np.zeros(562))
#     #     np.concatenate(np.array(Xs[i]),np.zeros(562))
#     #   # print(len(Xs[i]))
#     return np.array(Xs), np.array(ys).reshape(-1, 1)

In [ ]:
# xnew, ynew = create_dataset(
#     x,
#    y
# )
# #x.to_numpy()
# #xnew = x.to_numpy().reshape((x.shape[0],1,x.shape[1]))
# #ynew = y.to_numpy().reshape((y.shape[0],1))

# #print(x)

# # X_test, y_test = create_dataset(
# #    test_x,test_y,TIME_STEPS,
# #     STEP
# # )
# print(xnew.shape)
# print(ynew.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test  = train_test_split(x, y,shuffle=True, test_size=0.2, random_state=42)

In [ ]:
train_X_lstm = np.array(X_train).reshape(X_train.shape[0], 1, X_train.shape[1])
test_X_lstm = np.array(X_test).reshape(X_test.shape[0], 1, X_test.shape[1])
X_pred=np.array(test).reshape(test.shape[0], 1, test.shape[1])

In [ ]:
#optimization

from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0.00005,
    patience=9,
    verbose=1,
    restore_best_weights=True,
)
# learning rate 
lr_scheduler = ReduceLROnPlateau(
    monitor='val_acc',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)
callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
print(X_train.shape[1])
print(X_train.shape)


562
(5881, 562)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from keras.optimizers import RMSprop,Adam, SGD

model = Sequential()
model.add((LSTM(
          units=32, 
          input_shape=[X_train.shape[0], X_train.shape[1]], return_sequences=False,activation='relu'
      )
    )
)
model.add(Dropout(rate=0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 32)                76160     
_________________________________________________________________
dropout_44 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_88 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_89 (Dense)             (None, 6)                 198       
Total params: 77,414
Trainable params: 77,414
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    train_X_lstm, y_train,
    epochs=100,
    verbose =1,
    batch_size=64,
    validation_data=(test_X_lstm, y_test),
    shuffle=True,
    callbacks=callbacks,
)

Epoch 1/100
184/184 [==============================] - 1s 5ms/step - loss: 0.0509 - acc: 0.9815 - val_loss: 0.0532 - val_acc: 0.9810
Epoch 2/100
184/184 [==============================] - 1s 5ms/step - loss: 0.0497 - acc: 0.9803 - val_loss: 0.0717 - val_acc: 0.9701
Epoch 3/100
184/184 [==============================] - 1s 5ms/step - loss: 0.0459 - acc: 0.9821 - val_loss: 0.0604 - val_acc: 0.9803
Epoch 4/100
184/184 [==============================] - 1s 5ms/step - loss: 0.0427 - acc: 0.9828 - val_loss: 0.0634 - val_acc: 0.9755
Epoch 5/100
184/184 [==============================] - 1s 5ms/step - loss: 0.0420 - acc: 0.9838 - val_loss: 0.0519 - val_acc: 0.9816
Epoch 6/100
184/184 [==============================] - 1s 4ms/step - loss: 0.0427 - acc: 0.9830 - val_loss: 0.0506 - val_acc: 0.9796
Epoch 7/100
184/184 [==============================] - 1s 5ms/step - loss: 0.0456 - acc: 0.9827 - val_loss: 0.0630 - val_acc: 0.9776
Epoch 8/100
184/184 [==============================] - 1s 5ms/step - 

In [ ]:
ide = data["id"]
Xpred = data.drop(labels=[ "id"], axis = 1)
Xpred = np.array(Xpred).reshape(Xpred.shape[0], 1, Xpred.shape[1])
y_pred = model.predict(Xpred)
y_pred = np.argmax(y_pred, axis=1)
y_pred = y_pred+1
df = pd.concat([pd.Series(ide, name="id"), pd.Series(y_pred, name="activity")], axis = 1)
df.to_csv(f"pred18.csv", index = False)